In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# function to find the key of the value when label encoding the classes
def key_find(classes, pred):
    for key, val in classes.items():
        if pred == val:
            return key
        
        
label_encoder = LabelEncoder()

# get data and instantiate Random Forest Classifier
X = pd.read_csv("/Users/kritinshanmugam/Desktop/gene_expression.csv")
y = pd.read_csv("/Users/kritinshanmugam/Desktop/label.csv")

# Handle missing values
X = X.dropna()
y = y.dropna()

# Drop ID column
X.drop(columns=X.columns[0], axis=1)

# Standardize features
X = (X - X.mean()) / X.std()



In [ ]:
from sklearn.model_selection import train_test_split

# Drop the first column of cell ID
y.drop(columns=y.columns[0], axis=1)

# Create copy of original (untransformed) y data to compare later with predictions
y_orig = y.copy()

# One-hot encode the target variable
y = pd.DataFrame(label_encoder.fit_transform(y))

# Print the mapping between original classes and encoded integers
class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))


# Combine data sets for ease of splitting into train and test data sets
combined_XY = pd.concat([X, y], axis=1)

# Split the data into train and test sets
train, test = train_test_split(combined_XY, test_size=0.25, stratify=y['Time point'])

# Get the train/test data for the features
x_train = train.drop(columns=['Time Point', 'Annotation'], axis=1)
x_test = test.drop(columns=['Time Point', 'Annotation'], axis=1)

# Get the train/test data for the target 
y_train = train['Annotation']
y_test = test['Annotation']


In [ ]:
# Train the random forest model
classifier = RandomForestClassifier(random_state=31)

classifier.fit(x_train, y_train)

# Obtain the predictions
y_predictions = classifier.predict(x_test)

# parameters for the model
params = {
    'n_estimators': [200, 600, 1_000, 1_400, 1_800],
    'max_depth': [3, 5, 7, 10, 13],
    'min_sample_split': [2, 5, 8, 11, 14, 17],
    'min_sample_leaf' : [2, 4, 6, 8],
    'max_features': [2, 4, 6, 8]
}





# Instantiate an empty dataframe to store the predictions in pre-transform format
predicted = pd.DataFrame({'predicted_class': []})

# Loop through the predictions and transform them back from the binary state
for prediction in y_predictions:
    class_pred = key_find(class_mapping, prediction)
    predicted.loc[len(predicted)] = class_pred

# Create list to store rows that have mispredicted cell types
incorrect_list = []

for row in range(len(predicted)):
    if predicted.iloc[row, 0] != y_orig.iloc[row, 0]:
        incorrect_list.append(row)
        


In [ ]:
import shap

explaine = shap.TreeExplainer(model)
shap_values = explaine.shap_values(X)
row = 1
shap.waterfall_plot(shap.Explanation(values=shap_values[0][row],
                                     base_values=explaine.expected_value[0], data=x_test.iloc[row],
                                     feature_names=x_test.columns.tolist()))

shap.plots.waterfall(shap_values[0][row])